#### Setup

In [ ]:
!pip install -U tensorflow keras

In [ ]:
!pip install -U talos

In [ ]:
# Import general Python libraries
import pandas as pd
import numpy as np
import random
import sklearn
import seaborn as sns
import os
import matplotlib.pyplot as plt

In [ ]:
# Specify seeds for random-operations
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [ ]:
# Import sklearn-specific modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [ ]:
# Import tensorflow-specific modules
import tensorflow as tf
tf.random.set_seed(seed_value)
print("Tensorflow Version: {}".format(tf.__version__))
print("Keras Version: {}".format(tf.keras.__version__))

Tensorflow Version: 2.4.1
Keras Version: 2.4.0


In [ ]:
# Import keras-specific modules
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LayerNormalization, GaussianNoise, Activation
from tensorflow.keras.optimizers import Adadelta, RMSprop, Adam, Adamax, Nadam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.initializers import GlorotNormal, GlorotUniform, LecunNormal, LecunUniform, HeNormal, HeUniform
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.utils import plot_model

In [ ]:
# Import talos-specific modules
import talos

In [ ]:
# Set pandas options
pd.set_option("display.max_columns", None)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### a) Prepare Dataset

In [ ]:
# Import Dataset
kickstarter_df = pd.read_csv("04_Final Datasets/Kickstarter_Structured.csv", index_col=0)
print(kickstarter_df.shape)
print(len(kickstarter_df.index.unique()))
kickstarter_df.head(1)

(246891, 35)
246891


,campaign_successful,goal,number_of_collaborators,funding_period,days_between_created_and_launched,launch_quartal,staff_pick,campaign_has_demo_video,campaign_has_environmental_commitments,number_of_images,number_of_videos,number_of_audios,number_of_interactives,number_of_words,number_of_links,creator_verified_identity,creator_fb_auth,creator_has_image,creator_allows_follows,number_of_creator_backings,number_of_creator_projects,facebook_linked,twitter_linked,instagram_linked,linkedin_linked,number_of_rewards,number_of_words_per_reward,lowest_pledge_level,highest_pledge_level,has_limited_rewards,has_shipped_rewards,has_restricted_shipping_rewards,avg_months_until_reward,location,category
22821161,0,50000.0,0,45,102,3,0,1,0,13,0,0,0,1378,2,1,0,1,1,0,1,0,0,0,0,8,36.75,5.0,1417.0,0,1,1,5.5,United States,Design_Product Design


In [ ]:
# Convert categorical features into dummy-variables
print("Before: {}".format(kickstarter_df.shape))
kickstarter_df["launch_quartal"] = kickstarter_df.launch_quartal.apply(str)
kickstarter_df = pd.get_dummies(kickstarter_df, prefix=["launch_quartal", "location", "category"], columns=["launch_quartal", "location", "category"], drop_first=False)
print("After: {}".format(kickstarter_df.shape))
kickstarter_df.head()

Before: (246891, 35)
After: (246891, 234)


,campaign_successful,goal,number_of_collaborators,funding_period,days_between_created_and_launched,staff_pick,campaign_has_demo_video,campaign_has_environmental_commitments,number_of_images,number_of_videos,number_of_audios,number_of_interactives,number_of_words,number_of_links,creator_verified_identity,creator_fb_auth,creator_has_image,creator_allows_follows,number_of_creator_backings,number_of_creator_projects,facebook_linked,twitter_linked,instagram_linked,linkedin_linked,number_of_rewards,number_of_words_per_reward,lowest_pledge_level,highest_pledge_level,has_limited_rewards,has_shipped_rewards,has_restricted_shipping_rewards,avg_months_until_reward,launch_quartal_1,launch_quartal_2,launch_quartal_3,launch_quartal_4,location_Africa,location_Australia,location_Belgium,location_Canada,location_China,location_Denmark,location_France,location_Germany,location_Hong Kong,location_Ireland,location_Italy,location_Japan,location_Latin and South America,location_Mexico,location_Netherlands,location_New Zealand,location_No Location,location_Norway,location_Oceania and Antarctica,location_Rest of Asia,location_Rest of Europe,location_Singapore,location_Spain,location_Sweden,location_Switzerland,location_United Kingdom,location_United States,category_Art_Ceramics,category_Art_Conceptual Art,category_Art_Digital Art,category_Art_Illustration,category_Art_Installations,category_Art_Mixed Media,category_Art_No Subcategory,category_Art_Painting,category_Art_Performance Art,category_Art_Public Art,category_Art_Sculpture,category_Art_Social Practice,category_Art_Textiles,category_Art_Video Art,category_Comics_Anthologies,category_Comics_Comic Books,category_Comics_Events,category_Comics_Graphic Novels,category_Comics_No Subcategory,category_Comics_Webcomics,category_Crafts_Candles,category_Crafts_Crochet,category_Crafts_DIY,category_Crafts_Embroidery,category_Crafts_Glass,category_Crafts_Knitting,category_Crafts_No Subcategory,category_Crafts_Pottery,category_Crafts_Printing,category_Crafts_Quilts,category_Crafts_Stationery,category_Crafts_Taxidermy,category_Crafts_Weaving,category_Crafts_Woodworking,category_Dance_No Subcategory,category_Dance_Performances,category_Dance_Residencies,category_Dance_Spaces,category_Dance_Workshops,category_Design_Architecture,category_Design_Civic Design,category_Design_Graphic Design,category_Design_Interactive Design,category_Design_No Subcategory,category_Design_Product Design,category_Design_Toys,category_Design_Typography,category_Fashion_Accessories,category_Fashion_Apparel,category_Fashion_Childrenswear,category_Fashion_Couture,category_Fashion_Footwear,category_Fashion_Jewelry,category_Fashion_No Subcategory,category_Fashion_Pet Fashion,category_Fashion_Ready-to-wear,category_Film & Video_Action,category_Film & Video_Animation,category_Film & Video_Comedy,category_Film & Video_Documentary,category_Film & Video_Drama,category_Film & Video_Experimental,category_Film & Video_Family,category_Film & Video_Fantasy,category_Film & Video_Festivals,category_Film & Video_Horror,category_Film & Video_Movie Theaters,category_Film & Video_Music Videos,category_Film & Video_Narrative Film,category_Film & Video_No Subcategory,category_Film & Video_Romance,category_Film & Video_Science Fiction,category_Film & Video_Shorts,category_Film & Video_Television,category_Film & Video_Thrillers,category_Film & Video_Webseries,category_Food_Bacon,category_Food_Community Gardens,category_Food_Cookbooks,category_Food_Drinks,category_Food_Events,category_Food_Farmer's Markets,category_Food_Farms,category_Food_Food Trucks,category_Food_No Subcategory,category_Food_Restaurants,category_Food_Small Batch,category_Food_Spaces,category_Food_Vegan,category_Games_Gaming Hardware,category_Games_Live Games,category_Games_Mobile Games,category_Games_No Subcategory,category_Games_Playing Cards,category_Games_Puzzles,category_Games_Tabletop Games,category_Games_Video Games,category_Journalism_Audio,category_Journalism_No Subcategory,category_

In [ ]:
# Convert dataset and target variable to Numpy Arrays
y = kickstarter_df["campaign_successful"].to_numpy()
kickstarter_notarget_df = kickstarter_df.drop(columns=["campaign_successful"])
X = kickstarter_notarget_df.astype("float32").to_numpy()

print(type(y))
print(y.shape)
print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(246891,)
<class 'numpy.ndarray'>
(246891, 233)


In [ ]:
# Retrieve column names which will be used for later pre-processing
feature_names = kickstarter_notarget_df.columns.values
numeric_feature_names = ["goal", "number_of_collaborators", "funding_period", "days_between_created_and_launched", "number_of_images", "number_of_videos", "number_of_audios", "number_of_interactives", "number_of_words", "number_of_links", "number_of_creator_backings", "number_of_creator_projects", "number_of_rewards", "number_of_words_per_reward", "lowest_pledge_level", "highest_pledge_level", "avg_months_until_reward"]
binary_feature_names = [x for x in feature_names if x not in numeric_feature_names]
numeric_features = [kickstarter_notarget_df.columns.get_loc(x) for x in numeric_feature_names]
binary_features = [kickstarter_notarget_df.columns.get_loc(x) for x in binary_feature_names]

In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
X_subtrain, X_val, y_subtrain, y_val = train_test_split(X_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(X_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(X_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (209858, 233)
Shape of y_train: (209858,)
Shape of X_subtrain: (172824, 233)
Shape of y_subtrain: (172824,)
Shape of X_val: (37034, 233)
Shape of y_val: (37034,)
Shape of X_test: (37033, 233)
Shape of y_test: (37033,)


#### a) Baseline

This model provides the baseline, which will be used as starting point for the hyperparameter search. The goal is to gradually improve the model performance.

**Hyperparameters:**
- Untransformed Dataset
- Regularizer = None
- Optimizer = RMSprop_centered
- Weight Initialization = GlorotNormal
- Batch Size = 512
- Number of Hidden Layers = 1
- Number of Neurons = 233
- Activation Function = sigmoid

In [ ]:
model = Sequential()
model.add(Dense(233, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 1.1019 - binary_accuracy: 0.5965 - val_loss: 0.5333 - val_binary_accuracy: 0.7116
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5182 - binary_accuracy: 0.7265 - val_loss: 0.5231 - val_binary_accuracy: 0.7289
Epoch 3/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5130 - binary_accuracy: 0.7325 - val_loss: 0.5105 - val_binary_accuracy: 0.7382
Epoch 4/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5079 - binary_accuracy: 0.7364 - val_loss: 0.5082 - val_binary_accuracy: 0.7362
Epoch 5/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5047 - binary_accuracy: 0.7420 - val_loss: 0.5123 - val_binary_accuracy: 0.7303
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4988 - binary_accuracy: 0.7466 - val_loss: 0.4979 - val_binary_accuracy: 0.7465
Epoch 7/100
338/338 [==============================] - 2s 

#### b) Test Different Preprocessing Strategies

##### 1. Standardization: Numerical + Binary Attributes

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", StandardScaler(), numeric_features),
      ("binary", StandardScaler(), binary_features)
    ],
    remainder="drop", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-0.298547,-0.198447,-0.941790,-0.301747,-0.542916,-0.264041,-0.08432,-0.01557,-1.130397,-0.746823,-0.215401,-0.224846,-0.265058,-0.639590,-0.094083,-0.563342,0.613136,-0.287578,0.675031,-0.08672,-1.081998,-0.947352,0.147337,0.078146,-0.654237,-0.432702,-0.33426,-0.159431,-1.120286,-1.49029,-0.673015,1.766646,-0.594752,-0.607508,-0.541083,-0.033,-0.145087,-0.04121,-0.199623,-0.036584,-0.052331,-0.070387,-0.097759,-0.057777,-0.052387,-0.094291,-0.048765,-0.053432,-0.048346,-0.084526,-0.063451,-0.059955,-0.040139,-0.009622,-0.064318,-0.085869,-0.051829,-0.072474,-0.066504,-0.041839,-0.324079,0.561241,-0.033436,-0.054723,-0.069468,-0.101491,-0.038213,-0.092748,-0.155436,-0.097396,-0.077544,-0.09176,-0.072312,-0.011024,-0.031379,-0.025692,-0.036663,-0.096636,-0.021654,-0.075865,-0.109888,-0.044136,-0.038741,-0.022442,-0.059613,-0.019693,-0.020697,-0.022052,-0.111494,-0.017181,-0.027542,-0.012955,-0.026796,-0.004811,-0.015215,-0.060052,-0.08109,-0.055933,-0.014027,-0.025006,-0.021385,-0.045878,-0.030725,-0.072996,-0.034038,-0.092335,-0.223068,-0.012266,-0.015591,-0.104757,-0.150343,-0.038666,-0.028676,-0.053377,-0.064273,-0.144796,-0.020415,-0.054989,-0.045752,-0.082381,-0.07941,-0.196261,-0.079263,-0.039557,-0.032647,-0.03025,-0.028978,-0.061205,-0.028166,-0.044201,-0.113237,-0.162721,-0.024656,-0.043805,5.617208,-0.053649,-0.048645,-0.12157,-0.018944,-0.028978,-0.038891,-0.090241,-0.042459,-0.036663,-0.060439,-0.076173,-0.166301,-0.097909,-0.072916,-0.039336,-0.047128,-0.031655,-0.055568,-0.089324,-0.078221,-0.088331,-0.029078,-0.172891,-0.151786,-0.035539,-0.068622,-0.022442,-0.045624,-0.034038,-0.058578,-0.028371,-0.009001,-0.084317,-0.0125,-0.114203,-0.07728,-0.057727,-0.106438,-0.124334,-0.073833,-0.030058,-0.020273,-0.047128,-0.215988,-0.095994,-0.03082,-0.03941,-0.138353,-0.076938,-0.026579,-0.043406,-0.040067,-0.122254,-0.055881,-0.065887,-0.043473,-0.053812,-0.035293,-0.08456,-0.0322,-0.139306,-0.02152,-0.156196,-0.012955,-0.029572,-0.016317,-0.120561,-0.147604,-0.054936,-0.064364,-0.053541,-0.02152,-0.049416,-0.036105,-0.03963,-0.14633,-0.029767,-0.048105,-0.026579,-0.031928,-0.089619,-0.092685,-0.025006,-0.11784,-0.037832,-0.093192,-0.040284,-0.029177,-0.054403,-0.103594,-0.025579,-0.0322,-0.040067,-0.0322,-0.051037,-0.141534,-0.066985,-0.022183
1,-0.307414,-0.198447,-2.019196,-0.326723,-0.542916,-0.264041,-0.08432,-0.01557,-0.369989,0.009728,-0.028455,-0.224846,-0.636566,-0.636051,-0.067116,-0.563342,-0.513072,-0.287578,0.675031,-0.08672,-1.081998,1.055574,0.147337,0.078146,-0.654237,-0.432702,-0.33426,6.272299,-1.120286,0.67101,1.485850,1.766646,-0.594752,-0.607508,-0.541083,-0.033,-0.145087,-0.04121,-0.199623,-0.036584,-0.052331,-0.070387,-0.097759,-0.057777,-0.052387,-0.094291,-0.048765,-0.053432,-0.048346,-0.084526,-0.063451,-0.059955,-0.040139,-0.009622,-0.064318,-0.085869,-0.051829,-0.072474,-0.066504,-0.041839,-0.324079,0.561241,-0.033436,-0.054723,-0.069468,-0.101491,-0.038213,-0.092748,-0.155436,-0.097396,-0.077544,-0.09176,-0.072312,-0.011024,-0.031379,-0.025692,-0.036663,-0.096636,-0.021654,-0.075865,-0.109888,-0.044136,-0.038741,-0.022442,-0.059613,-0.019693,-0.020697,-0.022052,-0.111494,-0.017181,-

In [ ]:
model = Sequential()
model.add(Dense(233, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val), batch_size=512, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.5268 - binary_accuracy: 0.7303 - val_loss: 0.4801 - val_binary_accuracy: 0.7644
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4740 - binary_accuracy: 0.7677 - val_loss: 0.4710 - val_binary_accuracy: 0.7674
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4677 - binary_accuracy: 0.7717 - val_loss: 0.4662 - val_binary_accuracy: 0.7702
Epoch 4/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4636 - binary_accuracy: 0.7751 - val_loss: 0.4614 - val_binary_accuracy: 0.7727
Epoch 5/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4606 - binary_accuracy: 0.7756 - val_loss: 0.4593 - val_binary_accuracy: 0.7739
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4558 - binary_accuracy: 0.7776 - val_loss: 0.4566 - val_binary_accuracy: 0.7767
Epoch 7/100
338/338 [==============================] - 2s 

##### 2. Standardization: Only Numerical Attributes

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", StandardScaler(), numeric_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-0.298547,-0.198447,-0.941790,-0.301747,-0.542916,-0.264041,-0.08432,-0.01557,-1.130397,-0.746823,-0.215401,-0.224846,-0.265058,-0.639590,-0.094083,-0.563342,0.613136,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.307414,-0.198447,-2.019196,-0.326723,-0.542916,-0.264041,-0.08432,-0.01557,-0.369989,0.009728,-0.028455,-0.224846,-0.636566,-0.636051,-0.067116,-0.563342,-0.513072,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.288786,-0.198447,-0.326129,1.762865,0.762853,0.903184,-0.08432,-0.01557,3.482626,0.261912,1.168003,0.223308,-0.450812,0.610507,-0.094083,-0.587589,-0.400451,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = Sequential()
model.add(Dense(233, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val), batch_size=512, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.5443 - binary_accuracy: 0.7134 - val_loss: 0.4730 - val_binary_accuracy: 0.7684
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4690 - binary_accuracy: 0.7701 - val_loss: 0.4665 - val_binary_accuracy: 0.7679
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4635 - binary_accuracy: 0.7737 - val_loss: 0.4606 - val_binary_accuracy: 0.7737
Epoch 4/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4608 - binary_accuracy: 0.7761 - val_loss: 0.4571 - val_binary_accuracy: 0.7756
Epoch 5/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4586 - binary_accuracy: 0.7762 - val_loss: 0.4559 - val_binary_accuracy: 0.7754
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4546 - binary_accuracy: 0.7783 - val_loss: 0.4530 - val_binary_accuracy: 0.7776
Epoch 7/100
338/338 [==============================] - 2s 

Result:
- Standardization helped extremely to increase the performance of the NN
- Standardizing only the numerical features and leaving the binary as they are was the best strategy

##### 3. Normalization: Numerical + Binary Features

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", Pipeline([
                            ("normalization", StandardScaler()),
                            ("standarization", MinMaxScaler(feature_range=(-1,1)))
                          ]), numeric_features),
      ("binary", MinMaxScaler(feature_range=(-1,1)), binary_features)
    ],
    remainder="drop", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-0.999650,-1.0,-0.567010,-0.995506,-1.00000,-1.000000,-1.0,-1.0,-0.988894,-1.000000,-1.000000,-1.000000,-0.947137,-0.929240,-0.999815,-0.99200,-0.777778,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-0.999862,-1.0,-0.855670,-0.997432,-1.00000,-1.000000,-1.0,-1.0,-0.918719,-0.960265,-0.994299,-1.000000,-0.964758,-0.928947,-0.998155,-0.99200,-0.916667,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-0.999416,-1.0,-0.402062,-0.836276,-0.87027,-0.935484,-1.0,-1.0,-0.563174,-0.947020,-0.957811,-0.963636,-0.955947,-0.825965,-0.999815,-0.99712,-0.902778,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-

In [ ]:
model = Sequential()
model.add(Dense(233, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val), batch_size=512, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.6392 - binary_accuracy: 0.6553 - val_loss: 0.5821 - val_binary_accuracy: 0.6938
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5878 - binary_accuracy: 0.6947 - val_loss: 0.6031 - val_binary_accuracy: 0.6834
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5757 - binary_accuracy: 0.7017 - val_loss: 0.5656 - val_binary_accuracy: 0.7182
Epoch 4/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5694 - binary_accuracy: 0.7071 - val_loss: 0.5763 - val_binary_accuracy: 0.7090
Epoch 5/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5643 - binary_accuracy: 0.7126 - val_loss: 0.5449 - val_binary_accuracy: 0.7239
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5590 - binary_accuracy: 0.7137 - val_loss: 0.5406 - val_binary_accuracy: 0.7287
Epoch 7/100
338/338 [==============================] - 2s 

Result: Normalization is not helpful

##### 4. Power Transformation of Numerical Features

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", PowerTransformer(method="yeo-johnson", standardize=True), numeric_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-1.083699,-0.239021,-0.986135,-0.190185,-0.933752,-0.366213,-0.13909,-0.025692,-2.667854,-1.554894,-0.878420,-0.612750,-0.063636,-0.619896,-1.382878,-0.74978,0.866256,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.621048,-0.239021,-2.927746,-0.514813,-0.933752,-0.366213,-0.13909,-0.025692,-0.148796,0.467339,1.228547,-0.612750,-0.601676,-0.613763,0.471788,-0.74978,-0.472150,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.768322,-0.239021,-0.229063,1.802791,1.300685,2.727485,-0.13909,-0.025692,2.257514,0.751221,1.769156,1.758142,-0.313814,0.831888,-1.382878,-1.25173,-0.290671,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = Sequential()
model.add(Dense(233, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val), batch_size=512, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=7, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.5084 - binary_accuracy: 0.7359 - val_loss: 0.4293 - val_binary_accuracy: 0.7925
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4245 - binary_accuracy: 0.7943 - val_loss: 0.4293 - val_binary_accuracy: 0.7920
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4225 - binary_accuracy: 0.7967 - val_loss: 0.4260 - val_binary_accuracy: 0.7947
Epoch 4/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4227 - binary_accuracy: 0.7969 - val_loss: 0.4247 - val_binary_accuracy: 0.7937
Epoch 5/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4224 - binary_accuracy: 0.7965 - val_loss: 0.4243 - val_binary_accuracy: 0.7951
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4204 - binary_accuracy: 0.7984 - val_loss: 0.4238 - val_binary_accuracy: 0.7954
Epoch 7/100
338/338 [==============================] - 2s 

Result: Transforming Numerical Features into a Normal Distribution helps extremely

##### 5. Decorrelation with PCA

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", Pipeline([
                            ("powertransform", PowerTransformer(method="yeo-johnson", standardize=True)),
                            ("pca", PCA())
                          ]), numeric_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-2.601059,-0.410007,-0.404783,1.164287,-0.091849,-0.212521,-0.529216,2.294785,0.580161,0.953602,-0.148654,-0.430559,0.612913,0.851988,-0.351759,0.900483,0.139061,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.390127,-2.211286,0.641949,0.480664,-0.239196,-0.387141,2.109289,0.654540,-0.251707,-0.133599,-0.882627,0.938515,1.286549,-0.575849,0.198403,-0.318513,0.027164,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.884293,-2.409525,2.013652,-0.189625,0.385735,0.039777,-1.415718,0.185991,-2.201274,-0.502807,0.497177,-0.674013,0.354346,0.169585,1.200940,-1.191646,-0.339670,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = Sequential()
model.add(Dense(233, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val), batch_size=512, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=7, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4835 - binary_accuracy: 0.7559 - val_loss: 0.4296 - val_binary_accuracy: 0.7925
Epoch 2/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4247 - binary_accuracy: 0.7943 - val_loss: 0.4297 - val_binary_accuracy: 0.7918
Epoch 3/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4230 - binary_accuracy: 0.7963 - val_loss: 0.4270 - val_binary_accuracy: 0.7937
Epoch 4/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4235 - binary_accuracy: 0.7963 - val_loss: 0.4259 - val_binary_accuracy: 0.7929
Epoch 5/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4235 - binary_accuracy: 0.7960 - val_loss: 0.4255 - val_binary_accuracy: 0.7945
Epoch 6/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4214 - binary_accuracy: 0.7975 - val_loss: 0.4248 - val_binary_accuracy: 0.7955
Epoch 7/100
338/338 [==============================] - 2s 

Result: 
- De-correlated + power-transformed dataset was the best
- however, perform this transformation only for numerical attributes, not binary

##### Perform Final Dataset Transformation:

In [ ]:
# Strategy: PowerTransform + Standardize + Decorrelate (PCA) Numerical Features; Leave Binary Features as they are
preprocessing = ColumnTransformer(
    [
      ("numeric", Pipeline([
                            ("powertransform", PowerTransformer(method="yeo-johnson", standardize=True)),
                            ("pca", PCA())
                          ]), numeric_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain = preprocessing.transform(X_subtrain)
X_val = preprocessing.transform(X_val)

preprocessing2 = ColumnTransformer(
    [
      ("numeric", Pipeline([
                            ("powertransform", PowerTransformer(method="yeo-johnson", standardize=True)),
                            ("pca", PCA())
                          ]), numeric_features),
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_train)
X_train = preprocessing2.transform(X_train)
X_test = preprocessing2.transform(X_test)

In [ ]:
# Re-arrange feature_names, since they have been changed by the ColumnTransformer
feature_names = numeric_feature_names + binary_feature_names

In [ ]:
# Validation
tmp = pd.DataFrame(X_test, columns=feature_names)
print(tmp.shape)
tmp.head(3)

(37033, 233)


,goal,number_of_collaborators,funding_period,days_between_created_and_launched,number_of_images,number_of_videos,number_of_audios,number_of_interactives,number_of_words,number_of_links,number_of_creator_backings,number_of_creator_projects,number_of_rewards,number_of_words_per_reward,lowest_pledge_level,highest_pledge_level,avg_months_until_reward,staff_pick,campaign_has_demo_video,campaign_has_environmental_commitments,creator_verified_identity,creator_fb_auth,creator_has_image,creator_allows_follows,facebook_linked,twitter_linked,instagram_linked,linkedin_linked,has_limited_rewards,has_shipped_rewards,has_restricted_shipping_rewards,launch_quartal_1,launch_quartal_2,launch_quartal_3,launch_quartal_4,location_Africa,location_Australia,location_Belgium,location_Canada,location_China,location_Denmark,location_France,location_Germany,location_Hong Kong,location_Ireland,location_Italy,location_Japan,location_Latin and South America,location_Mexico,location_Netherlands,location_New Zealand,location_No Location,location_Norway,location_Oceania and Antarctica,location_Rest of Asia,location_Rest of Europe,location_Singapore,location_Spain,location_Sweden,location_Switzerland,location_United Kingdom,location_United States,category_Art_Ceramics,category_Art_Conceptual Art,category_Art_Digital Art,category_Art_Illustration,category_Art_Installations,category_Art_Mixed Media,category_Art_No Subcategory,category_Art_Painting,category_Art_Performance Art,category_Art_Public Art,category_Art_Sculpture,category_Art_Social Practice,category_Art_Textiles,category_Art_Video Art,category_Comics_Anthologies,category_Comics_Comic Books,category_Comics_Events,category_Comics_Graphic Novels,category_Comics_No Subcategory,category_Comics_Webcomics,category_Crafts_Candles,category_Crafts_Crochet,category_Crafts_DIY,category_Crafts_Embroidery,category_Crafts_Glass,category_Crafts_Knitting,category_Crafts_No Subcategory,category_Crafts_Pottery,category_Crafts_Printing,category_Crafts_Quilts,category_Crafts_Stationery,category_Crafts_Taxidermy,category_Crafts_Weaving,category_Crafts_Woodworking,category_Dance_No Subcategory,category_Dance_Performances,category_Dance_Residencies,category_Dance_Spaces,category_Dance_Workshops,category_Design_Architecture,category_Design_Civic Design,category_Design_Graphic Design,category_Design_Interactive Design,category_Design_No Subcategory,category_Design_Product Design,category_Design_Toys,category_Design_Typography,category_Fashion_Accessories,category_Fashion_Apparel,category_Fashion_Childrenswear,category_Fashion_Couture,category_Fashion_Footwear,category_Fashion_Jewelry,category_Fashion_No Subcategory,category_Fashion_Pet Fashion,category_Fashion_Ready-to-wear,category_Film & Video_Action,category_Film & Video_Animation,category_Film & Video_Comedy,category_Film & Video_Documentary,category_Film & Video_Drama,category_Film & Video_Experimental,category_Film & Video_Family,category_Film & Video_Fantasy,category_Film & Video_Festivals,category_Film & Video_Horror,category_Film & Video_Movie Theaters,category_Film & Video_Music Videos,category_Film & Video_Narrative Film,category_Film & Video_No Subcategory,category_Film & Video_Romance,category_Film & Video_Science Fiction,category_Film & Video_Shorts,category_Film & Video_Television,category_Film & Video_Thrillers,category_Film & Video_Webseries,category_Food_Bacon,category_Food_Community Gardens,category_Food_Cookbooks,category_Food_Drinks,category_Food_Events,category_Food_Farmer's Markets,category_Food_Farms,category_Food_Food Trucks,category_Food_No Subcategory,category_Food_Restaurants,category_Food_Small Batch,category_Food_Spaces,category_Food_Vegan,category_Games_Gaming Hardware,category_Games_Live Games,category_Games_Mobile Games,category_Games_No Subcategory,category_Games_Playing Cards,category_Games_Puzzles,category_Games_Tabletop Games,category_Games_Video Games,category_Journalism_Audio,category_Journalism_No Subcategory,category_Journalism_Photo,cat

#### c) Test Different Model Hyperparameters

##### New Baseline:

In [ ]:
model = Sequential()
model.add(Dense(233, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=7, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.4835 - binary_accuracy: 0.7559 - val_loss: 0.4296 - val_binary_accuracy: 0.7925
Epoch 2/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4247 - binary_accuracy: 0.7943 - val_loss: 0.4297 - val_binary_accuracy: 0.7918
Epoch 3/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4230 - binary_accuracy: 0.7963 - val_loss: 0.4270 - val_binary_accuracy: 0.7937
Epoch 4/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4235 - binary_accuracy: 0.7963 - val_loss: 0.4259 - val_binary_accuracy: 0.7929
Epoch 5/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4235 - binary_accuracy: 0.7960 - val_loss: 0.4255 - val_binary_accuracy: 0.7945
Epoch 6/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4214 - binary_accuracy: 0.7975 - val_loss: 0.4248 - val_binary_accuracy: 0.7955
Epoch 7/100
338/338 [==============================] - 2s 

Fixed Hyperparameters:
- Dataset Transformation = PowerTransform + PCA Numerical Features; No Changes at Binary Features
- Loss Function = Binary Crossentropy (Recommended for Binary Classification)
- Learning Rate / Learning Rate Annealing = not necessary to tune, since I am using only adaptive learning rate optimizers
- Number of Epochs = tuned via Early Stopping

Tunable Hyperparameters:
- Optimizers = RMSprop, RMSprop_centered, Adam, Adam_amsgrad, Adamax, Nadam
- Mini-Batch Size = can be tuned independently from other hyperparameters and then be held fixed, since it mainly affects the convergence time and not the predictive performance (Bengio 2012)
- Regularizer = L1, L2 (L2 penalizes larger weights more strongly, whereas L1 penalizes lower weights more strongly; L1 performs automatic feature selection)
- Regularization Rate (start with defaul and increase/decrease by factor 10)
- Weight Initializer = typically, a clipped-off initializer, such as Glorot, is used
- Number of Layers
- Number of Hidden Units
- Activation Function
- Different Types of Regularization (e.g. Dropout, Batch Normalization, Gaussian Noise, Layer Normalization)


##### Perform Random Search - Iteration 1 To Get First Insights Which Hyperparameter Combinations Perform Best:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()
  if regularization=="L2":
     reg_rate = params["reg_rate"]
  else:
    reg_rate=0

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, activation=activation_function, kernel_initializer=init, kernel_regularizer=L2(reg_rate)))
    
    if regularization=="dropout":
      model.add(Dropout(rate=0.5, seed=seed_value))

    if regularization=="batch_normalization":
      model.add(BatchNormalization())
    
    if regularization=="layer_normalization":
      model.add(LayerNormalization())
    
    if regularization=="gaussian_noise":
      model.add(GaussianNoise(stddev=1))

    if network_shape=="triangle":
      number_of_units = number_of_units * network_decay

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=L2(l2=reg_rate), kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [1, 2, 3, 5, 10],
    "number_of_units_first_layer" : [5, 10, 30, 50, 100, 150, 233, 300, 500],
    "network_shape" : ["brick", "triangle"],
    "network_decay" : [0.3, 0.5, 0.7],
    "activation_function" : ["sigmoid", "tanh", "relu", "elu", "gelu", "selu", "swish"],
    "regularization" : ["L2", "dropout", "batch_normalization", "layer_normalization", "gaussian_noise"],
    "optimizer" : ["RMSprop", "RMSprop_centered", "Adam", "Adam_amsgrad", "Adamax", "Nadam"],
    "reg_rate" : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    "weight_initializer" : ["glorot_normal", "glorot_uniform", "he_normal", "he_uniform", "lecun_normal", "lecun_uniform"],
    "batch_size" : [128, 256, 512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params, fraction_limit=0.00001,experiment_name="test1", seed=seed_value, print_params=True)



  0%|          | 0/71 [00:00<?, ?it/s]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'RMSprop_centered', 'reg_rate': 0.01, 'regularization': 'dropout', 'weight_initializer': 'glorot_uniform'}
Epoch 1/100
338/338 [==============================] - 17s 46ms/step - loss: 0.4571 - binary_accuracy: 0.7715 - val_loss: 0.4219 - val_binary_accuracy: 0.7965
Epoch 2/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4230 - binary_accuracy: 0.7965 - val_loss: 0.4179 - val_binary_accuracy: 0.7991
Epoch 3/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4185 - binary_accuracy: 0.8010 - val_loss: 0.4145 - val_binary_accuracy: 0.8009
Epoch 4/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4160 - binary_accuracy: 0.8021 - val_loss: 0.4117 - val_binary_accuracy: 0.8005
Epoch 5/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4149



  1%|▏         | 1/71 [04:47<5:35:19, 287.43s/it]

{'activation_function': 'sigmoid', 'batch_size': 128, 'network_decay': 0.5, 'network_shape': 'triangle', 'number_of_layers': 2, 'number_of_units_first_layer': 5, 'optimizer': 'RMSprop', 'reg_rate': 0.01, 'regularization': 'L2', 'weight_initializer': 'glorot_normal'}
Epoch 1/100
1351/1351 [==============================] - 3s 2ms/step - loss: 0.6878 - binary_accuracy: 0.6496 - val_loss: 0.6571 - val_binary_accuracy: 0.6501
Epoch 2/100
1351/1351 [==============================] - 2s 2ms/step - loss: 0.6531 - binary_accuracy: 0.6534 - val_loss: 0.6504 - val_binary_accuracy: 0.6501
Epoch 3/100
1351/1351 [==============================] - 2s 2ms/step - loss: 0.6491 - binary_accuracy: 0.6512 - val_loss: 0.6485 - val_binary_accuracy: 0.6501
Epoch 4/100
1351/1351 [==============================] - 2s 2ms/step - loss: 0.6481 - binary_accuracy: 0.6503 - val_loss: 0.6478 - val_binary_accuracy: 0.6501
Epoch 5/100
1351/1351 [==============================] - 2s 2ms/step - loss: 0.6480 - binary_accu



  3%|▎         | 2/71 [05:01<3:56:16, 205.45s/it]

...

{'activation_function': 'sigmoid', 'batch_size': 512, 'network_decay': 0.5, 'network_shape': 'brick', 'number_of_layers': 10, 'number_of_units_first_layer': 150, 'optimizer': 'Nadam', 'reg_rate': 0.01, 'regularization': 'layer_normalization', 'weight_initializer': 'glorot_normal'}
Epoch 1/100
338/338 [==============================] - 29s 72ms/step - loss: 0.4849 - binary_accuracy: 0.7641 - val_loss: 0.4288 - val_binary_accuracy: 0.7941
Epoch 2/100
338/338 [==============================] - 23s 68ms/step - loss: 0.4204 - binary_accuracy: 0.7968 - val_loss: 0.4633 - val_binary_accuracy: 0.7818
Epoch 3/100
338/338 [==============================] - 24s 71ms/step - loss: 0.4163 - binary_accuracy: 0.8006 - val_loss: 0.4183 - val_binary_accuracy: 0.7981
Epoch 4/100
338/338 [==============================] - 24s 70ms/step - loss: 0.4123 - binary_accuracy: 0.8025 - val_loss: 0.4189 - val_binary_accuracy: 0.7939
Epoch 5/100
338/338 [==============================] - 23s 69ms/step - loss: 0.409



 46%|████▋     | 33/71 [1:16:24<1:22:13, 129.83s/it]

{'activation_function': 'selu', 'batch_size': 512, 'network_decay': 0.3, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 30, 'optimizer': 'Adamax', 'reg_rate': 100, 'regularization': 'gaussian_noise', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 2s 4ms/step - loss: 0.8429 - binary_accuracy: 0.5925 - val_loss: 0.4805 - val_binary_accuracy: 0.7649
Epoch 2/100
338/338 [==============================] - 1s 4ms/step - loss: 0.6171 - binary_accuracy: 0.6712 - val_loss: 0.4679 - val_binary_accuracy: 0.7698
Epoch 3/100
338/338 [==============================] - 1s 4ms/step - loss: 0.5562 - binary_accuracy: 0.7003 - val_loss: 0.4576 - val_binary_accuracy: 0.7777
Epoch 4/100
338/338 [==============================] - 1s 4ms/step - loss: 0.5152 - binary_accuracy: 0.7294 - val_loss: 0.4512 - val_binary_accuracy: 0.7837
Epoch 5/100
338/338 [==============================] - 1s 4ms/step - loss: 0.4883 - binary_accuracy:



 48%|████▊     | 34/71 [1:17:08<1:04:12, 104.11s/it]

{'activation_function': 'elu', 'batch_size': 128, 'network_decay': 0.5, 'network_shape': 'triangle', 'number_of_layers': 10, 'number_of_units_first_layer': 50, 'optimizer': 'Nadam', 'reg_rate': 10, 'regularization': 'layer_normalization', 'weight_initializer': 'lecun_normal'}
Epoch 1/100


In [ ]:
# Get search results for all evaluated hyperparameter combinations sorted by val_accuracy
analyze_object = talos.Analyze(search_object)
analyze_object.data[["duration", "loss", "binary_accuracy", "val_loss", "val_binary_accuracy", "optimizer", "regularizer", "reg_rate", "weight_initializer", "batch_size", "epochs"]]\
.sort_values(by="val_binary_accuracy", ascending=False).round(4)

Result: 
- batch_size = 256 & 512 provided best results -> drop 128
- number_of_layers = best models had layer of 3 & 10 -> not sure which value can be dropped
- number_of_units = 5, 10, & 30 gave bad results; best models had high number of units (i.e. 100 - 500) -> keep 50, 100, 150, 233, 500
- activation = gelu, relu, elu, selu provided best results (maybe also keep swish)
- dropout, layer_normalization, and batch_normalization gave best results
- network shape = either brick or triangle with low decay (e.g. 0.7) provided good results

##### Perform Random Search Iteration 2 with Smaller Hyperparameter Space:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()
  if regularization=="L2":
     reg_rate = params["reg_rate"]
  else:
    reg_rate=0

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, activation=activation_function, kernel_initializer=init, kernel_regularizer=L2(reg_rate)))
    
    if regularization=="dropout":
      model.add(Dropout(rate=0.5, seed=seed_value))

    if regularization=="batch_normalization":
      model.add(BatchNormalization())
    
    if regularization=="layer_normalization":
      model.add(LayerNormalization())

    if network_shape=="triangle":
      number_of_units = round(number_of_units * network_decay)

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=L2(l2=reg_rate), kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [1, 2, 3, 5],
    "number_of_units_first_layer" : [100, 150, 233, 300, 500],
    "network_shape" : ["brick", "triangle"],
    "network_decay" : [0.5, 0.7],
    "activation_function" : ["relu", "elu", "gelu", "selu", "swish"],
    "regularization" : ["dropout", "batch_normalization", "layer_normalization"],
    "optimizer" : ["RMSprop", "RMSprop_centered", "Adam", "Adam_amsgrad"],
    "weight_initializer" : ["glorot_normal", "glorot_uniform", "he_normal", "he_uniform", "lecun_normal", "lecun_uniform"],
    "batch_size" : [256, 512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params, fraction_limit=0.001,experiment_name="test1", seed=seed_value, print_params=True)

  0%|          | 0/57 [00:00<?, ?it/s]

{'activation_function': 'swish', 'batch_size': 256, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 300, 'optimizer': 'RMSprop_centered', 'regularization': 'layer_normalization', 'weight_initializer': 'glorot_uniform'}
Epoch 1/100
676/676 [==============================] - 15s 20ms/step - loss: 0.4669 - binary_accuracy: 0.7765 - val_loss: 0.4348 - val_binary_accuracy: 0.7838
Epoch 2/100
676/676 [==============================] - 13s 20ms/step - loss: 0.4117 - binary_accuracy: 0.8019 - val_loss: 0.4287 - val_binary_accuracy: 0.7973
Epoch 3/100
676/676 [==============================] - 13s 19ms/step - loss: 0.4033 - binary_accuracy: 0.8085 - val_loss: 0.4390 - val_binary_accuracy: 0.7946
Epoch 4/100
676/676 [==============================] - 13s 20ms/step - loss: 0.3994 - binary_accuracy: 0.8092 - val_loss: 0.4215 - val_binary_accuracy: 0.7993
Epoch 5/100
676/676 [==============================] - 13s 19ms/step - loss: 0.3942 - bi

  2%|▏         | 1/57 [02:14<2:05:19, 134.28s/it]

{'activation_function': 'swish', 'batch_size': 512, 'network_decay': 0.5, 'network_shape': 'triangle', 'number_of_layers': 5, 'number_of_units_first_layer': 100, 'optimizer': 'RMSprop_centered', 'regularization': 'batch_normalization', 'weight_initializer': 'glorot_uniform'}
Epoch 1/100
338/338 [==============================] - 6s 11ms/step - loss: 0.4845 - binary_accuracy: 0.7656 - val_loss: 0.4233 - val_binary_accuracy: 0.7982
Epoch 2/100
338/338 [==============================] - 3s 10ms/step - loss: 0.4106 - binary_accuracy: 0.8025 - val_loss: 0.4117 - val_binary_accuracy: 0.8006
Epoch 3/100
338/338 [==============================] - 3s 10ms/step - loss: 0.4034 - binary_accuracy: 0.8087 - val_loss: 0.4132 - val_binary_accuracy: 0.7992
Epoch 4/100
338/338 [==============================] - 3s 9ms/step - loss: 0.4021 - binary_accuracy: 0.8077 - val_loss: 0.4106 - val_binary_accuracy: 0.7999
Epoch 5/100
338/338 [==============================] - 3s 9ms/step - loss: 0.3987 - binary_ac

  4%|▎         | 2/57 [02:53<1:37:02, 105.86s/it]

...

 96%|█████████▋| 55/57 [2:02:55<05:32, 166.29s/it]

{'activation_function': 'selu', 'batch_size': 512, 'network_decay': 0.5, 'network_shape': 'brick', 'number_of_layers': 2, 'number_of_units_first_layer': 500, 'optimizer': 'RMSprop', 'regularization': 'batch_normalization', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 14s 39ms/step - loss: 0.5295 - binary_accuracy: 0.7575 - val_loss: 0.4442 - val_binary_accuracy: 0.7837
Epoch 2/100
338/338 [==============================] - 13s 39ms/step - loss: 0.4200 - binary_accuracy: 0.7957 - val_loss: 0.4206 - val_binary_accuracy: 0.7965
Epoch 3/100
338/338 [==============================] - 13s 38ms/step - loss: 0.4093 - binary_accuracy: 0.8045 - val_loss: 0.4207 - val_binary_accuracy: 0.7936
Epoch 4/100
338/338 [==============================] - 13s 38ms/step - loss: 0.4072 - binary_accuracy: 0.8052 - val_loss: 0.4165 - val_binary_accuracy: 0.7984
Epoch 5/100
338/338 [==============================] - 13s 39ms/step - loss: 0.4038 - binary_accuracy: 0.

 98%|█████████▊| 56/57 [2:05:32<02:43, 163.45s/it]

{'activation_function': 'relu', 'batch_size': 256, 'network_decay': 0.5, 'network_shape': 'brick', 'number_of_layers': 1, 'number_of_units_first_layer': 150, 'optimizer': 'RMSprop', 'regularization': 'dropout', 'weight_initializer': 'he_normal'}
Epoch 1/100
676/676 [==============================] - 4s 5ms/step - loss: 0.4809 - binary_accuracy: 0.7569 - val_loss: 0.4199 - val_binary_accuracy: 0.7975
Epoch 2/100
676/676 [==============================] - 3s 4ms/step - loss: 0.4226 - binary_accuracy: 0.7960 - val_loss: 0.4150 - val_binary_accuracy: 0.7998
Epoch 3/100
676/676 [==============================] - 3s 4ms/step - loss: 0.4159 - binary_accuracy: 0.8025 - val_loss: 0.4145 - val_binary_accuracy: 0.8019
Epoch 4/100
676/676 [==============================] - 3s 4ms/step - loss: 0.4145 - binary_accuracy: 0.8033 - val_loss: 0.4149 - val_binary_accuracy: 0.8010
Epoch 5/100
676/676 [==============================] - 3s 4ms/step - loss: 0.4133 - binary_accuracy: 0.8037 - val_loss: 0.4118

100%|██████████| 57/57 [2:06:26<00:00, 133.09s/it]


In [ ]:
# Get search results for all evaluated hyperparameter combinations sorted by val_accuracy
analyze_object = talos.Analyze(search_object)
analyze_object.data.sort_values(by="val_binary_accuracy", ascending=False).round(4).head(5)

,start,end,duration,round_epochs,loss,binary_accuracy,val_loss,val_binary_accuracy,activation_function,batch_size,network_decay,network_shape,number_of_layers,number_of_units_first_layer,optimizer,regularization,weight_initializer
41,01/22/21-144240,01/22/21-144757,317.1675,23,0.3924,0.8137,0.4044,0.8060,gelu,512,0.7,triangle,3,500,Adam,dropout,he_uniform
46,01/22/21-151003,01/22/21-151232,149.3489,18,0.3928,0.8150,0.4102,0.8044,relu,256,0.5,brick,3,233,Adam_amsgrad,dropout,he_normal
31,01/22/21-142729,01/22/21-142840,70.9462,18,0.4040,0.8092,0.4091,0.8040,gelu,256,0.5,brick,1,233,RMSprop,dropout,he_normal
50,01/22/21-151710,01/22/21-151827,76.9696,16,0.4044,0.8080,0.4102,0.8036,gelu,256,0.7,brick,1,300,RMSprop,dropout,glorot_uniform
20,01/22/21-140711,01/22/21-140939,147.5310,16,0.3877,0.8172,0.4071,0.8035,relu,512,0.7,brick,3,300,Adam_amsgrad,dropout,glorot_uniform


Result:
- batch_size = 512 performed better -> drop 256
- gelu & relu had the highest performance
- no significant difference between triangle and brick shape
- between 1 and 3 layers provided the best results
- number of units didn't influence the performance so strongly anymore; however, best model had 500 units
- Adam performed a little bit better than RMSprop
- Dropout was the best regularizer; layer-norm. was a little bit better than batch-norm.

##### Fine-Tune The So-Far Best Found Model:

###### New Baseline:

In [ ]:
# New Baseline which needs to be improved
model = Sequential()
model.add(Dense(500, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))
model.add(Dense(350, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))
model.add(Dense(245, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeUniform(seed=seed_value)))
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val),batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 15s 42ms/step - loss: 0.4913 - binary_accuracy: 0.7510 - val_loss: 0.4187 - val_binary_accuracy: 0.7971
Epoch 2/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4272 - binary_accuracy: 0.7928 - val_loss: 0.4139 - val_binary_accuracy: 0.7997
Epoch 3/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4190 - binary_accuracy: 0.7998 - val_loss: 0.4146 - val_binary_accuracy: 0.8010
Epoch 4/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4167 - binary_accuracy: 0.8005 - val_loss: 0.4103 - val_binary_accuracy: 0.8015
Epoch 5/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4147 - binary_accuracy: 0.8013 - val_loss: 0.4092 - val_binary_accuracy: 0.8021
Epoch 6/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4116 - binary_accuracy: 0.8035 - val_loss: 0.4081 - val_binary_accuracy: 0.8032
Epoch 7/100
338/338 [=========================

###### Test different types of regularizers:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()
  if regularization=="L2":
     reg_rate = params["reg_rate"]
  else:
    reg_rate=0

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, activation=activation_function, kernel_initializer=init, kernel_regularizer=L2(reg_rate)))
    
    if regularization=="dropout":
      model.add(Dropout(rate=0.5, seed=seed_value))

    if regularization=="batch_normalization":
      model.add(BatchNormalization())
    
    if regularization=="layer_normalization":
      model.add(LayerNormalization())
    
    if regularization=="gaussian_noise":
      model.add(GaussianNoise(stddev=0.1))

    if network_shape=="triangle":
      number_of_units = round(number_of_units * network_decay)

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=L2(l2=reg_rate), kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [3],
    "number_of_units_first_layer" : [500],
    "network_shape" : ["triangle"],
    "network_decay" : [0.7],
    "activation_function" : ["gelu"],
    "regularization" : ["dropout", "batch_normalization", "layer_normalization", "gaussian_noise"],
    "reg_rate" : [0.01],
    "optimizer" : ["Adam"],
    "weight_initializer" : ["he_uniform"],
    "batch_size" : [512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params,experiment_name="test1", seed=seed_value, print_params=True)

  0%|          | 0/4 [00:00<?, ?it/s]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 15s 41ms/step - loss: 0.4913 - binary_accuracy: 0.7510 - val_loss: 0.4187 - val_binary_accuracy: 0.7971
Epoch 2/100
338/338 [==============================] - 13s 40ms/step - loss: 0.4272 - binary_accuracy: 0.7928 - val_loss: 0.4139 - val_binary_accuracy: 0.7997
Epoch 3/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4190 - binary_accuracy: 0.7998 - val_loss: 0.4146 - val_binary_accuracy: 0.8010
Epoch 4/100
338/338 [==============================] - 14s 40ms/step - loss: 0.4167 - binary_accuracy: 0.8005 - val_loss: 0.4103 - val_binary_accuracy: 0.8015
Epoch 5/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4147 - binary_accura

 25%|██▌       | 1/4 [05:17<15:51, 317.19s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'batch_normalization', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 16s 42ms/step - loss: 0.4903 - binary_accuracy: 0.7700 - val_loss: 0.4175 - val_binary_accuracy: 0.8000
Epoch 2/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4029 - binary_accuracy: 0.8076 - val_loss: 0.4137 - val_binary_accuracy: 0.8007
Epoch 3/100
338/338 [==============================] - 14s 42ms/step - loss: 0.3918 - binary_accuracy: 0.8153 - val_loss: 0.4165 - val_binary_accuracy: 0.7980
Epoch 4/100
338/338 [==============================] - 14s 42ms/step - loss: 0.3857 - binary_accuracy: 0.8181 - val_loss: 0.4170 - val_binary_accuracy: 0.7965
Epoch 5/100
338/338 [==============================] - 14s 42ms/step - loss: 0.3768 - b

 50%|█████     | 2/4 [06:58<08:24, 252.46s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'layer_normalization', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 21s 58ms/step - loss: 0.4661 - binary_accuracy: 0.7758 - val_loss: 0.4136 - val_binary_accuracy: 0.8002
Epoch 2/100
338/338 [==============================] - 19s 57ms/step - loss: 0.4046 - binary_accuracy: 0.8066 - val_loss: 0.4106 - val_binary_accuracy: 0.8022
Epoch 3/100
338/338 [==============================] - 19s 57ms/step - loss: 0.3947 - binary_accuracy: 0.8129 - val_loss: 0.4125 - val_binary_accuracy: 0.7994
Epoch 4/100
338/338 [==============================] - 19s 57ms/step - loss: 0.3892 - binary_accuracy: 0.8146 - val_loss: 0.4150 - val_binary_accuracy: 0.7982
Epoch 5/100
338/338 [==============================] - 19s 57ms/step - loss: 0.3810 - b

 75%|███████▌  | 3/4 [09:15<03:37, 217.80s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'gaussian_noise', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 16s 44ms/step - loss: 0.4531 - binary_accuracy: 0.7723 - val_loss: 0.4132 - val_binary_accuracy: 0.7997
Epoch 2/100
338/338 [==============================] - 15s 43ms/step - loss: 0.4111 - binary_accuracy: 0.8019 - val_loss: 0.4083 - val_binary_accuracy: 0.8027
Epoch 3/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4035 - binary_accuracy: 0.8075 - val_loss: 0.4088 - val_binary_accuracy: 0.8027
Epoch 4/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4004 - binary_accuracy: 0.8091 - val_loss: 0.4091 - val_binary_accuracy: 0.8010
Epoch 5/100
338/338 [==============================] - 15s 44ms/step - loss: 0.3969 - binary

100%|██████████| 4/4 [12:13<00:00, 183.31s/it]


In [ ]:
# Get search results for all evaluated hyperparameter combinations sorted by val_accuracy
analyze_object = talos.Analyze(search_object)
analyze_object.data.sort_values(by="val_binary_accuracy", ascending=False).round(4)

,start,end,duration,round_epochs,loss,binary_accuracy,val_loss,val_binary_accuracy,activation_function,batch_size,network_decay,network_shape,number_of_layers,number_of_units_first_layer,optimizer,reg_rate,regularization,weight_initializer
0,01/22/21-165147,01/22/21-165704,317.0095,23,0.3924,0.8137,0.4044,0.8060,gelu,512,0.7,triangle,3,500,Adam,0.01,dropout,he_uniform
3,01/22/21-170102,01/22/21-170400,177.4552,12,0.3504,0.8370,0.4279,0.7989,gelu,512,0.7,triangle,3,500,Adam,0.01,gaussian_noise,he_uniform
2,01/22/21-165845,01/22/21-170102,136.6777,7,0.3588,0.8314,0.4337,0.7960,gelu,512,0.7,triangle,3,500,Adam,0.01,layer_normalization,he_uniform
1,01/22/21-165704,01/22/21-165845,101.2021,7,0.3529,0.8374,0.4352,0.7921,gelu,512,0.7,triangle,3,500,Adam,0.01,batch_normalization,he_uniform


Result:
- dropout performed by far the best
- but: also Gaussian noise was promising

###### Test if regularizers before the activation can improve results:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()
  if regularization=="L2":
     reg_rate = params["reg_rate"]
  else:
    reg_rate=0

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, kernel_initializer=init, kernel_regularizer=L2(reg_rate)))
    
    if regularization=="dropout":
      model.add(Dropout(rate=0.5, seed=seed_value))

    if regularization=="batch_normalization":
      model.add(BatchNormalization())
    
    if regularization=="layer_normalization":
      model.add(LayerNormalization())
    
    if regularization=="gaussian_noise":
      model.add(GaussianNoise(stddev=0.1))

    model.add(Activation(activation=activation_function))

    if network_shape=="triangle":
      number_of_units = round(number_of_units * network_decay)

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=L2(l2=reg_rate), kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [3],
    "number_of_units_first_layer" : [500],
    "network_shape" : ["triangle"],
    "network_decay" : [0.7],
    "activation_function" : ["gelu"],
    "regularization" : ["dropout", "batch_normalization", "layer_normalization", "gaussian_noise"],
    "reg_rate" : [0.01],
    "optimizer" : ["Adam"],
    "weight_initializer" : ["he_uniform"],
    "batch_size" : [512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params,experiment_name="test1", seed=seed_value, print_params=True)

  0%|          | 0/4 [00:00<?, ?it/s]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 15s 42ms/step - loss: 0.5115 - binary_accuracy: 0.7403 - val_loss: 0.4407 - val_binary_accuracy: 0.7902
Epoch 2/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4287 - binary_accuracy: 0.7916 - val_loss: 0.4331 - val_binary_accuracy: 0.7960
Epoch 3/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4177 - binary_accuracy: 0.8005 - val_loss: 0.4254 - val_binary_accuracy: 0.7961
Epoch 4/100
338/338 [==============================] - 14s 40ms/step - loss: 0.4139 - binary_accuracy: 0.8033 - val_loss: 0.4408 - val_binary_accuracy: 0.7989
Epoch 5/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4107 - binary_accura

 25%|██▌       | 1/4 [03:01<09:04, 181.60s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'batch_normalization', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 17s 45ms/step - loss: 0.4582 - binary_accuracy: 0.7758 - val_loss: 0.4146 - val_binary_accuracy: 0.7983
Epoch 2/100
338/338 [==============================] - 15s 43ms/step - loss: 0.4014 - binary_accuracy: 0.8084 - val_loss: 0.4111 - val_binary_accuracy: 0.8002
Epoch 3/100
338/338 [==============================] - 15s 43ms/step - loss: 0.3899 - binary_accuracy: 0.8162 - val_loss: 0.4116 - val_binary_accuracy: 0.8021
Epoch 4/100
338/338 [==============================] - 15s 43ms/step - loss: 0.3834 - binary_accuracy: 0.8190 - val_loss: 0.4126 - val_binary_accuracy: 0.7995
Epoch 5/100
338/338 [==============================] - 15s 43ms/step - loss: 0.3752 - b

 50%|█████     | 2/4 [05:00<05:25, 162.85s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'layer_normalization', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 21s 59ms/step - loss: 0.4632 - binary_accuracy: 0.7708 - val_loss: 0.4124 - val_binary_accuracy: 0.8005
Epoch 2/100
338/338 [==============================] - 20s 58ms/step - loss: 0.4042 - binary_accuracy: 0.8060 - val_loss: 0.4109 - val_binary_accuracy: 0.8006
Epoch 3/100
338/338 [==============================] - 20s 59ms/step - loss: 0.3947 - binary_accuracy: 0.8129 - val_loss: 0.4093 - val_binary_accuracy: 0.8013
Epoch 4/100
338/338 [==============================] - 20s 58ms/step - loss: 0.3886 - binary_accuracy: 0.8149 - val_loss: 0.4122 - val_binary_accuracy: 0.7989
Epoch 5/100
338/338 [==============================] - 20s 59ms/step - loss: 0.3817 - b

 75%|███████▌  | 3/4 [08:21<02:54, 174.18s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'gaussian_noise', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 16s 45ms/step - loss: 0.4501 - binary_accuracy: 0.7727 - val_loss: 0.4121 - val_binary_accuracy: 0.8003
Epoch 2/100
338/338 [==============================] - 15s 45ms/step - loss: 0.4085 - binary_accuracy: 0.8031 - val_loss: 0.4084 - val_binary_accuracy: 0.8022
Epoch 3/100
338/338 [==============================] - 15s 45ms/step - loss: 0.4012 - binary_accuracy: 0.8088 - val_loss: 0.4086 - val_binary_accuracy: 0.8025
Epoch 4/100
338/338 [==============================] - 15s 45ms/step - loss: 0.3979 - binary_accuracy: 0.8103 - val_loss: 0.4088 - val_binary_accuracy: 0.8009
Epoch 5/100
338/338 [==============================] - 15s 45ms/step - loss: 0.3941 - binary

100%|██████████| 4/4 [11:23<00:00, 170.90s/it]


In [ ]:
# Get search results for all evaluated hyperparameter combinations sorted by val_accuracy
analyze_object = talos.Analyze(search_object)
analyze_object.data.sort_values(by="val_binary_accuracy", ascending=False).round(4)

,start,end,duration,round_epochs,loss,binary_accuracy,val_loss,val_binary_accuracy,activation_function,batch_size,network_decay,network_shape,number_of_layers,number_of_units_first_layer,optimizer,reg_rate,regularization,weight_initializer
0,01/22/21-170718,01/22/21-171019,181.4161,13,0.3923,0.8143,0.4712,0.8017,gelu,512,0.7,triangle,3,500,Adam,0.01,dropout,he_uniform
3,01/22/21-171539,01/22/21-171841,182.0131,12,0.3346,0.8445,0.4449,0.7945,gelu,512,0.7,triangle,3,500,Adam,0.01,gaussian_noise,he_uniform
1,01/22/21-171019,01/22/21-171218,118.8821,8,0.3364,0.8455,0.4480,0.7904,gelu,512,0.7,triangle,3,500,Adam,0.01,batch_normalization,he_uniform
2,01/22/21-171218,01/22/21-171539,200.3671,10,0.3147,0.8559,0.4669,0.7874,gelu,512,0.7,triangle,3,500,Adam,0.01,layer_normalization,he_uniform


Result:
- performing regularization before the activation decreased performance
- only for Gaussian noise the performance increased

###### Test if Gaussian Noise + Dropout can increase performance:

In [ ]:
# New Baseline which needs to be improved
model = Sequential()
model.add(GaussianNoise(stddev=0.1))
model.add(Dropout(rate=0.2, seed=seed_value))

model.add(Dense(500, kernel_initializer=HeUniform(seed=seed_value)))
model.add(GaussianNoise(stddev=0.1))
model.add(Activation(activation="gelu"))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(350, kernel_initializer=HeUniform(seed=seed_value)))
model.add(GaussianNoise(stddev=0.1))
model.add(Activation(activation="gelu"))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(245, kernel_initializer=HeUniform(seed=seed_value)))
model.add(GaussianNoise(stddev=0.1))
model.add(Activation(activation="gelu"))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(1, activation="sigmoid", kernel_initializer=HeUniform(seed=seed_value)))

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val),batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 19s 55ms/step - loss: 0.5533 - binary_accuracy: 0.7125 - val_loss: 0.4429 - val_binary_accuracy: 0.7908
Epoch 2/100
338/338 [==============================] - 18s 55ms/step - loss: 0.4898 - binary_accuracy: 0.7544 - val_loss: 0.4398 - val_binary_accuracy: 0.7919
Epoch 3/100
338/338 [==============================] - 19s 56ms/step - loss: 0.4854 - binary_accuracy: 0.7565 - val_loss: 0.4409 - val_binary_accuracy: 0.7949
Epoch 4/100
338/338 [==============================] - 19s 55ms/step - loss: 0.4817 - binary_accuracy: 0.7579 - val_loss: 0.4276 - val_binary_accuracy: 0.7939
Epoch 5/100
338/338 [==============================] - 18s 54ms/step - loss: 0.4806 - binary_accuracy: 0.7600 - val_loss: 0.4373 - val_binary_accuracy: 0.7962
Epoch 6/100
338/338 [==============================] - 18s 54ms/step - loss: 0.4775 - binary_accuracy: 0.7631 - val_loss: 0.4310 - val_binary_accuracy: 0.7953
Epoch 7/100
338/338 [=========================

Result: it didn't help

###### Test if adding dropout to the input layer helps:

In [ ]:
model = Sequential()
model.add(Dropout(rate=0.2, seed=seed_value))

model.add(Dense(500, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(350, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(245, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(1, activation="sigmoid", kernel_initializer=HeUniform(seed=seed_value)))

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val),batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 15s 43ms/step - loss: 0.5393 - binary_accuracy: 0.7216 - val_loss: 0.4322 - val_binary_accuracy: 0.7957
Epoch 2/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4746 - binary_accuracy: 0.7642 - val_loss: 0.4283 - val_binary_accuracy: 0.7976
Epoch 3/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4702 - binary_accuracy: 0.7668 - val_loss: 0.4311 - val_binary_accuracy: 0.7982
Epoch 4/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4659 - binary_accuracy: 0.7707 - val_loss: 0.4199 - val_binary_accuracy: 0.7991
Epoch 5/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4654 - binary_accuracy: 0.7693 - val_loss: 0.4276 - val_binary_accuracy: 0.8008
Epoch 6/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4632 - binary_accuracy: 0.7722 - val_loss: 0.4212 - val_binary_accuracy: 0.7993
Epoch 7/100
338/338 [=========================

Result: only add dropout to the hidden layers, but not the input layer

###### Test if adding Gaussian noise just to the input layer helps:

In [ ]:
model = Sequential()
model.add(GaussianNoise(stddev=0.1))

model.add(Dense(500, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(350, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(245, activation="gelu", kernel_initializer=HeUniform(seed=seed_value)))
model.add(Dropout(rate=0.5, seed=seed_value))

model.add(Dense(1, activation="sigmoid", kernel_initializer=HeUniform(seed=seed_value)))

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val),batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 16s 44ms/step - loss: 0.5029 - binary_accuracy: 0.7448 - val_loss: 0.4265 - val_binary_accuracy: 0.7915
Epoch 2/100
338/338 [==============================] - 15s 43ms/step - loss: 0.4438 - binary_accuracy: 0.7830 - val_loss: 0.4215 - val_binary_accuracy: 0.7955
Epoch 3/100
338/338 [==============================] - 15s 43ms/step - loss: 0.4365 - binary_accuracy: 0.7879 - val_loss: 0.4228 - val_binary_accuracy: 0.7964
Epoch 4/100
338/338 [==============================] - 14s 43ms/step - loss: 0.4341 - binary_accuracy: 0.7885 - val_loss: 0.4160 - val_binary_accuracy: 0.7996
Epoch 5/100
338/338 [==============================] - 15s 43ms/step - loss: 0.4310 - binary_accuracy: 0.7911 - val_loss: 0.4172 - val_binary_accuracy: 0.7989
Epoch 6/100
338/338 [==============================] - 15s 43ms/step - loss: 0.4291 - binary_accuracy: 0.7933 - val_loss: 0.4144 - val_binary_accuracy: 0.8000
Epoch 7/100
338/338 [=========================

Result: it didn't help, i.e. keep only dropout at hidden layers

###### Test if weight decay helps:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()
  reg_rate = params["reg_rate"]

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, activation=activation_function, kernel_initializer=init, kernel_regularizer=L2(reg_rate)))
    
    if regularization=="dropout":
      model.add(Dropout(rate=0.5, seed=seed_value))

    if regularization=="batch_normalization":
      model.add(BatchNormalization())
    
    if regularization=="layer_normalization":
      model.add(LayerNormalization())
    
    if regularization=="gaussian_noise":
      model.add(GaussianNoise(stddev=0.1))

    if network_shape=="triangle":
      number_of_units = round(number_of_units * network_decay)

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=L2(l2=reg_rate), kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [3],
    "number_of_units_first_layer" : [500],
    "network_shape" : ["triangle"],
    "network_decay" : [0.7],
    "activation_function" : ["gelu"],
    "regularization" : ["dropout"],
    "reg_rate" : [0.001, 0.01, 1],
    "optimizer" : ["Adam"],
    "weight_initializer" : ["he_uniform"],
    "batch_size" : [512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params,experiment_name="test1", seed=seed_value, print_params=True)

  0%|          | 0/3 [00:00<?, ?it/s]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.001, 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 16s 44ms/step - loss: 1.6334 - binary_accuracy: 0.7504 - val_loss: 0.5421 - val_binary_accuracy: 0.7890
Epoch 2/100
338/338 [==============================] - 14s 42ms/step - loss: 0.5148 - binary_accuracy: 0.7872 - val_loss: 0.4671 - val_binary_accuracy: 0.7902
Epoch 3/100
338/338 [==============================] - 15s 43ms/step - loss: 0.4676 - binary_accuracy: 0.7911 - val_loss: 0.4597 - val_binary_accuracy: 0.7906
Epoch 4/100
338/338 [==============================] - 15s 45ms/step - loss: 0.4618 - binary_accuracy: 0.7909 - val_loss: 0.4576 - val_binary_accuracy: 0.7902
Epoch 5/100
338/338 [==============================] - 15s 45ms/step - loss: 0.4605 - binary_accur

 33%|███▎      | 1/3 [02:46<05:32, 166.41s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 0.01, 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 17s 46ms/step - loss: 8.0264 - binary_accuracy: 0.7335 - val_loss: 0.5678 - val_binary_accuracy: 0.7693
Epoch 2/100
338/338 [==============================] - 15s 45ms/step - loss: 0.5707 - binary_accuracy: 0.7660 - val_loss: 0.5541 - val_binary_accuracy: 0.7722
Epoch 3/100
338/338 [==============================] - 15s 45ms/step - loss: 0.5641 - binary_accuracy: 0.7650 - val_loss: 0.5498 - val_binary_accuracy: 0.7717
Epoch 4/100
338/338 [==============================] - 14s 42ms/step - loss: 0.5629 - binary_accuracy: 0.7628 - val_loss: 0.5454 - val_binary_accuracy: 0.7724
Epoch 5/100
338/338 [==============================] - 15s 44ms/step - loss: 0.5614 - binary_accura

 67%|██████▋   | 2/3 [05:11<02:40, 160.11s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'reg_rate': 1, 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 15s 41ms/step - loss: 712.9702 - binary_accuracy: 0.6174 - val_loss: 0.7074 - val_binary_accuracy: 0.6501
Epoch 2/100
338/338 [==============================] - 14s 41ms/step - loss: 0.6651 - binary_accuracy: 0.6533 - val_loss: 0.6492 - val_binary_accuracy: 0.6501
Epoch 3/100
338/338 [==============================] - 14s 41ms/step - loss: 0.6480 - binary_accuracy: 0.6513 - val_loss: 0.6476 - val_binary_accuracy: 0.6501
Epoch 4/100
338/338 [==============================] - 14s 42ms/step - loss: 0.6474 - binary_accuracy: 0.6503 - val_loss: 0.6474 - val_binary_accuracy: 0.6501
Epoch 5/100
338/338 [==============================] - 14s 42ms/step - loss: 0.6477 - binary_accurac

100%|██████████| 3/3 [06:36<00:00, 132.24s/it]


Result: adding weight decay significantly reduced the performance of the network (i.e. keep dropout as regularization technique)

###### Tune the dropout rate:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  dropout_rate = params["dropout_rate"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, activation=activation_function, kernel_initializer=init))
    
    if regularization=="dropout":
      model.add(Dropout(rate=dropout_rate, seed=seed_value))

    if network_shape=="triangle":
      number_of_units = round(number_of_units * network_decay)

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [3],
    "number_of_units_first_layer" : [500],
    "network_shape" : ["triangle"],
    "network_decay" : [0.7],
    "activation_function" : ["gelu"],
    "regularization" : ["dropout"],
    "dropout_rate" : [0.4, 0.5, 0.6],
    "optimizer" : ["Adam"],
    "weight_initializer" : ["he_uniform"],
    "batch_size" : [512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params,experiment_name="test1", seed=seed_value, print_params=True)


  0%|          | 0/3 [00:00<?, ?it/s]

{'activation_function': 'gelu', 'batch_size': 512, 'dropout_rate': 0.4, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 15s 42ms/step - loss: 0.4752 - binary_accuracy: 0.7615 - val_loss: 0.4162 - val_binary_accuracy: 0.7988
Epoch 2/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4219 - binary_accuracy: 0.7954 - val_loss: 0.4120 - val_binary_accuracy: 0.8007
Epoch 3/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4143 - binary_accuracy: 0.8018 - val_loss: 0.4123 - val_binary_accuracy: 0.8025
Epoch 4/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4124 - binary_accuracy: 0.8017 - val_loss: 0.4103 - val_binary_accuracy: 0.8022
Epoch 5/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4106 - binary_acc


 33%|███▎      | 1/3 [03:01<06:03, 181.79s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'dropout_rate': 0.5, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 15s 42ms/step - loss: 0.4913 - binary_accuracy: 0.7510 - val_loss: 0.4187 - val_binary_accuracy: 0.7971
Epoch 2/100
338/338 [==============================] - 14s 40ms/step - loss: 0.4272 - binary_accuracy: 0.7928 - val_loss: 0.4139 - val_binary_accuracy: 0.7997
Epoch 3/100
338/338 [==============================] - 14s 40ms/step - loss: 0.4190 - binary_accuracy: 0.7998 - val_loss: 0.4146 - val_binary_accuracy: 0.8010
Epoch 4/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4167 - binary_accuracy: 0.8005 - val_loss: 0.4103 - val_binary_accuracy: 0.8015
Epoch 5/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4147 - binary_acc


 67%|██████▋   | 2/3 [08:20<03:42, 222.89s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'dropout_rate': 0.6, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 14s 40ms/step - loss: 0.5199 - binary_accuracy: 0.7361 - val_loss: 0.4223 - val_binary_accuracy: 0.7946
Epoch 2/100
338/338 [==============================] - 13s 39ms/step - loss: 0.4350 - binary_accuracy: 0.7899 - val_loss: 0.4165 - val_binary_accuracy: 0.7992
Epoch 3/100
338/338 [==============================] - 13s 39ms/step - loss: 0.4256 - binary_accuracy: 0.7971 - val_loss: 0.4162 - val_binary_accuracy: 0.7998
Epoch 4/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4218 - binary_accuracy: 0.7974 - val_loss: 0.4125 - val_binary_accuracy: 0.8009
Epoch 5/100
338/338 [==============================] - 13s 40ms/step - loss: 0.4191 - binary_acc


100%|██████████| 3/3 [15:55<00:00, 318.35s/it]


Result: default rate of 0.5 is the best dropout rate

###### Test different activation functions:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, activation=activation_function, kernel_initializer=init))
    
    if regularization=="dropout":
      model.add(Dropout(rate=0.5, seed=seed_value))

    if network_shape=="triangle":
      number_of_units = round(number_of_units * network_decay)

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [3],
    "number_of_units_first_layer" : [500],
    "network_shape" : ["triangle"],
    "network_decay" : [0.7],
    "activation_function" : ["sigmoid", "tanh", "relu", "elu", "selu", "gelu", "swish"],
    "regularization" : ["dropout"],
    "optimizer" : ["Adam"],
    "weight_initializer" : ["he_uniform"],
    "batch_size" : [512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params,experiment_name="test1", seed=seed_value, print_params=True)


  0%|          | 0/7 [00:00<?, ?it/s]

{'activation_function': 'sigmoid', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 13s 36ms/step - loss: 0.5985 - binary_accuracy: 0.6774 - val_loss: 0.4334 - val_binary_accuracy: 0.7901
Epoch 2/100
338/338 [==============================] - 12s 36ms/step - loss: 0.4451 - binary_accuracy: 0.7809 - val_loss: 0.4278 - val_binary_accuracy: 0.7931
Epoch 3/100
338/338 [==============================] - 12s 36ms/step - loss: 0.4355 - binary_accuracy: 0.7879 - val_loss: 0.4274 - val_binary_accuracy: 0.7942
Epoch 4/100
338/338 [==============================] - 12s 36ms/step - loss: 0.4324 - binary_accuracy: 0.7914 - val_loss: 0.4242 - val_binary_accuracy: 0.7939
Epoch 5/100
338/338 [==============================] - 12s 36ms/step - loss: 0.4303 - binary_accuracy: 0.7920 - va


 14%|█▍        | 1/7 [07:29<44:59, 449.90s/it]

{'activation_function': 'tanh', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 13s 35ms/step - loss: 0.5367 - binary_accuracy: 0.7375 - val_loss: 0.4342 - val_binary_accuracy: 0.7924
Epoch 2/100
338/338 [==============================] - 12s 35ms/step - loss: 0.4413 - binary_accuracy: 0.7852 - val_loss: 0.4278 - val_binary_accuracy: 0.7929
Epoch 3/100
338/338 [==============================] - 12s 36ms/step - loss: 0.4331 - binary_accuracy: 0.7902 - val_loss: 0.4220 - val_binary_accuracy: 0.7941
Epoch 4/100
338/338 [==============================] - 12s 36ms/step - loss: 0.4306 - binary_accuracy: 0.7922 - val_loss: 0.4234 - val_binary_accuracy: 0.7970
Epoch 5/100
338/338 [==============================] - 12s 35ms/step - loss: 0.4282 - binary_accuracy: 0.7934 - val_l


 29%|██▊       | 2/7 [12:43<34:05, 409.09s/it]

...


 71%|███████▏  | 5/7 [25:09<10:14, 307.38s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 15s 41ms/step - loss: 0.4913 - binary_accuracy: 0.7510 - val_loss: 0.4187 - val_binary_accuracy: 0.7971
Epoch 2/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4272 - binary_accuracy: 0.7928 - val_loss: 0.4139 - val_binary_accuracy: 0.7997
Epoch 3/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4190 - binary_accuracy: 0.7998 - val_loss: 0.4146 - val_binary_accuracy: 0.8010
Epoch 4/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4167 - binary_accuracy: 0.8005 - val_loss: 0.4103 - val_binary_accuracy: 0.8015
Epoch 5/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4147 - binary_accuracy: 0.8013 - val_l


 86%|████████▌ | 6/7 [30:31<05:11, 311.75s/it]

{'activation_function': 'swish', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adam', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 14s 39ms/step - loss: 0.4841 - binary_accuracy: 0.7558 - val_loss: 0.4209 - val_binary_accuracy: 0.7965
Epoch 2/100
338/338 [==============================] - 13s 38ms/step - loss: 0.4289 - binary_accuracy: 0.7923 - val_loss: 0.4165 - val_binary_accuracy: 0.7978
Epoch 3/100
338/338 [==============================] - 13s 38ms/step - loss: 0.4219 - binary_accuracy: 0.7988 - val_loss: 0.4173 - val_binary_accuracy: 0.7993
Epoch 4/100
338/338 [==============================] - 13s 38ms/step - loss: 0.4194 - binary_accuracy: 0.7991 - val_loss: 0.4130 - val_binary_accuracy: 0.8002
Epoch 5/100
338/338 [==============================] - 13s 38ms/step - loss: 0.4179 - binary_accuracy: 0.7988 - val_


100%|██████████| 7/7 [33:20<00:00, 285.81s/it]


Result: gelu was the best; relu was the second-best

###### Tune different optimizers:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, activation=activation_function, kernel_initializer=init))
    
    if regularization=="dropout":
      model.add(Dropout(rate=0.5, seed=seed_value))

    if network_shape=="triangle":
      number_of_units = round(number_of_units * network_decay)

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [3],
    "number_of_units_first_layer" : [500],
    "network_shape" : ["triangle"],
    "network_decay" : [0.7],
    "activation_function" : ["gelu"],
    "regularization" : ["dropout"],
    "optimizer" : ["RMSprop", "RMSprop_centered", "Adam", "Adam_amsgrad", "Adamax", "Nadam"],
    "weight_initializer" : ["he_uniform"],
    "batch_size" : [512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params,experiment_name="test1", seed=seed_value, print_params=True)


  0%|          | 0/6 [00:00<?, ?it/s]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'RMSprop', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 15s 42ms/step - loss: 0.4850 - binary_accuracy: 0.7576 - val_loss: 0.4217 - val_binary_accuracy: 0.7966
Epoch 2/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4268 - binary_accuracy: 0.7942 - val_loss: 0.4148 - val_binary_accuracy: 0.8002
Epoch 3/100
338/338 [==============================] - 14s 42ms/step - loss: 0.4192 - binary_accuracy: 0.8004 - val_loss: 0.4126 - val_binary_accuracy: 0.8006
Epoch 4/100
338/338 [==============================] - 14s 43ms/step - loss: 0.4169 - binary_accuracy: 0.8008 - val_loss: 0.4106 - val_binary_accuracy: 0.8013
Epoch 5/100
338/338 [==============================] - 14s 43ms/step - loss: 0.4149 - binary_accuracy: 0.8024 - va


 17%|█▋        | 1/6 [06:30<32:32, 390.53s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'RMSprop_centered', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 16s 45ms/step - loss: 0.4823 - binary_accuracy: 0.7592 - val_loss: 0.4214 - val_binary_accuracy: 0.7968
Epoch 2/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4257 - binary_accuracy: 0.7945 - val_loss: 0.4148 - val_binary_accuracy: 0.8005
Epoch 3/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4187 - binary_accuracy: 0.8006 - val_loss: 0.4125 - val_binary_accuracy: 0.8009
Epoch 4/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4163 - binary_accuracy: 0.8014 - val_loss: 0.4103 - val_binary_accuracy: 0.8016
Epoch 5/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4145 - binary_accuracy: 0.


 33%|███▎      | 2/6 [13:15<26:18, 394.75s/it]

...


 67%|██████▋   | 4/6 [25:39<12:55, 387.77s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Adamax', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 15s 41ms/step - loss: 0.5127 - binary_accuracy: 0.7377 - val_loss: 0.4291 - val_binary_accuracy: 0.7903
Epoch 2/100
338/338 [==============================] - 14s 40ms/step - loss: 0.4451 - binary_accuracy: 0.7821 - val_loss: 0.4222 - val_binary_accuracy: 0.7947
Epoch 3/100
338/338 [==============================] - 13s 40ms/step - loss: 0.4341 - binary_accuracy: 0.7897 - val_loss: 0.4184 - val_binary_accuracy: 0.7971
Epoch 4/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4292 - binary_accuracy: 0.7920 - val_loss: 0.4161 - val_binary_accuracy: 0.7982
Epoch 5/100
338/338 [==============================] - 14s 41ms/step - loss: 0.4262 - binary_accuracy: 0.7948 - val


 83%|████████▎ | 5/6 [30:56<06:06, 366.61s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 3, 'number_of_units_first_layer': 500, 'optimizer': 'Nadam', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 16s 45ms/step - loss: 0.4943 - binary_accuracy: 0.7496 - val_loss: 0.4191 - val_binary_accuracy: 0.7973
Epoch 2/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4273 - binary_accuracy: 0.7928 - val_loss: 0.4136 - val_binary_accuracy: 0.8003
Epoch 3/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4193 - binary_accuracy: 0.7999 - val_loss: 0.4122 - val_binary_accuracy: 0.8005
Epoch 4/100
338/338 [==============================] - 15s 44ms/step - loss: 0.4165 - binary_accuracy: 0.8006 - val_loss: 0.4102 - val_binary_accuracy: 0.8019
Epoch 5/100
338/338 [==============================] - 15s 43ms/step - loss: 0.4145 - binary_accuracy: 0.8013 - val_


100%|██████████| 6/6 [37:20<00:00, 373.39s/it]


Result: RMSprop and RMSprop_centered performed best; Adam_amsgrad second

###### Perform last random search with different network architectures:

In [ ]:
# Define the Neural Network
def neural_network(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  number_of_layers = params["number_of_layers"]
  number_of_units = params["number_of_units_first_layer"]
  network_shape = params["network_shape"]
  network_decay = params["network_decay"]
  activation_function = params["activation_function"]
  regularization = params["regularization"]
  optimizer = params["optimizer"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()

  # Define Model
  model = Sequential()

  # Add hidden layers
  for i in range(0, number_of_layers):
    model.add(Dense(number_of_units, activation=activation_function, kernel_initializer=init))
    
    if regularization=="dropout":
      model.add(Dropout(rate=0.5, seed=seed_value))

    if network_shape=="triangle":
      number_of_units = round(number_of_units * network_decay)

  # Add output layer
  model.add(Dense(1, activation="sigmoid", kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", patience=5, restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "number_of_layers" : [1, 2, 3, 5],
    "number_of_units_first_layer" : [30, 50, 100, 200, 250, 500, 1000],
    "network_shape" : ["triangle", "brick"],
    "network_decay" : [0.3, 0.5, 0.7],
    "activation_function" : ["gelu"],
    "regularization" : ["dropout"],
    "optimizer" : ["RMSprop"],
    "weight_initializer" : ["he_uniform"],
    "batch_size" : [512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=neural_network, params=params, fraction_limit=0.5, experiment_name="test1", seed=seed_value, print_params=True)

  0%|          | 0/84 [00:00<?, ?it/s]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.5, 'network_shape': 'brick', 'number_of_layers': 3, 'number_of_units_first_layer': 30, 'optimizer': 'RMSprop', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 3s 5ms/step - loss: 0.5801 - binary_accuracy: 0.6871 - val_loss: 0.4406 - val_binary_accuracy: 0.7916
Epoch 2/100
338/338 [==============================] - 1s 4ms/step - loss: 0.4613 - binary_accuracy: 0.7814 - val_loss: 0.4365 - val_binary_accuracy: 0.7947
Epoch 3/100
338/338 [==============================] - 1s 4ms/step - loss: 0.4480 - binary_accuracy: 0.7890 - val_loss: 0.4544 - val_binary_accuracy: 0.7949
Epoch 4/100
338/338 [==============================] - 1s 4ms/step - loss: 0.4410 - binary_accuracy: 0.7911 - val_loss: 0.4594 - val_binary_accuracy: 0.7950
Epoch 5/100
338/338 [==============================] - 1s 4ms/step - loss: 0.4358 - binary_accuracy: 0.7935 - val_loss: 0.4761

  1%|          | 1/84 [00:16<22:31, 16.29s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.5, 'network_shape': 'brick', 'number_of_layers': 5, 'number_of_units_first_layer': 100, 'optimizer': 'RMSprop', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 5s 13ms/step - loss: 0.5757 - binary_accuracy: 0.6965 - val_loss: 0.4475 - val_binary_accuracy: 0.7913
Epoch 2/100
338/338 [==============================] - 4s 12ms/step - loss: 0.4547 - binary_accuracy: 0.7846 - val_loss: 0.4468 - val_binary_accuracy: 0.7959
Epoch 3/100
338/338 [==============================] - 4s 12ms/step - loss: 0.4417 - binary_accuracy: 0.7928 - val_loss: 0.4672 - val_binary_accuracy: 0.7971
Epoch 4/100
338/338 [==============================] - 4s 12ms/step - loss: 0.4359 - binary_accuracy: 0.7944 - val_loss: 0.4800 - val_binary_accuracy: 0.7979
Epoch 5/100
338/338 [==============================] - 4s 12ms/step - loss: 0.4310 - binary_accuracy: 0.7959 - val_loss: 

  2%|▏         | 2/84 [01:46<52:41, 38.56s/it]

...

 98%|█████████▊| 82/84 [3:21:55<04:35, 137.55s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'triangle', 'number_of_layers': 1, 'number_of_units_first_layer': 250, 'optimizer': 'RMSprop', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.5073 - binary_accuracy: 0.7453 - val_loss: 0.4216 - val_binary_accuracy: 0.7971
Epoch 2/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4233 - binary_accuracy: 0.7963 - val_loss: 0.4162 - val_binary_accuracy: 0.7988
Epoch 3/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4170 - binary_accuracy: 0.8003 - val_loss: 0.4136 - val_binary_accuracy: 0.8002
Epoch 4/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4153 - binary_accuracy: 0.8019 - val_loss: 0.4130 - val_binary_accuracy: 0.8000
Epoch 5/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4133 - binary_accuracy: 0.8027 - val_loss: 0.

 99%|█████████▉| 83/84 [3:22:34<01:48, 108.11s/it]

{'activation_function': 'gelu', 'batch_size': 512, 'network_decay': 0.7, 'network_shape': 'brick', 'number_of_layers': 3, 'number_of_units_first_layer': 200, 'optimizer': 'RMSprop', 'regularization': 'dropout', 'weight_initializer': 'he_uniform'}
Epoch 1/100
338/338 [==============================] - 6s 16ms/step - loss: 0.5045 - binary_accuracy: 0.7456 - val_loss: 0.4232 - val_binary_accuracy: 0.7951
Epoch 2/100
338/338 [==============================] - 5s 15ms/step - loss: 0.4337 - binary_accuracy: 0.7920 - val_loss: 0.4166 - val_binary_accuracy: 0.7984
Epoch 3/100
338/338 [==============================] - 5s 16ms/step - loss: 0.4250 - binary_accuracy: 0.7983 - val_loss: 0.4155 - val_binary_accuracy: 0.7990
Epoch 4/100
338/338 [==============================] - 5s 15ms/step - loss: 0.4224 - binary_accuracy: 0.7985 - val_loss: 0.4141 - val_binary_accuracy: 0.7996
Epoch 5/100
338/338 [==============================] - 5s 15ms/step - loss: 0.4203 - binary_accuracy: 0.8013 - val_loss: 

100%|██████████| 84/84 [3:24:15<00:00, 145.89s/it]


In [ ]:
# Get search results for all evaluated hyperparameter combinations sorted by val_accuracy
analyze_object = talos.Analyze(search_object)
analyze_object.data.sort_values(by="val_binary_accuracy", ascending=False).round(4).head(20)

,start,end,duration,round_epochs,loss,binary_accuracy,val_loss,val_binary_accuracy,activation_function,batch_size,network_decay,network_shape,number_of_layers,number_of_units_first_layer,optimizer,regularization,weight_initializer
5,01/22/21-215730,01/22/21-220102,212.7177,24,0.3934,0.8145,0.4040,0.8064,gelu,512,0.7,triangle,2,500,RMSprop,dropout,he_uniform
78,01/23/21-010329,01/23/21-010923,354.1986,16,0.3956,0.8142,0.4110,0.8059,gelu,512,0.5,triangle,3,1000,RMSprop,dropout,he_uniform
4,01/22/21-215518,01/22/21-215730,131.1640,26,0.4021,0.8114,0.4054,0.8055,gelu,512,0.7,triangle,3,250,RMSprop,dropout,he_uniform
47,01/22/21-234511,01/22/21-234821,190.1974,41,0.3936,0.8147,0.4048,0.8054,gelu,512,0.7,brick,2,250,RMSprop,dropout,he_uniform
22,01/22/21-224049,01/22/21-224302,133.2169,40,0.4069,0.8126,0.4067,0.8053,gelu,512,0.3,triangle,3,250,RMSprop,dropout,he_uniform
49,01/22/21-234914,01/23/21-000005,651.0824,22,0.3853,0.8184,0.4087,0.8053,gelu,512,0.7,brick,2,1000,RMSprop,dropout,he_uniform
43,01/22/21-233327,01/22/21-234336,609.6853,22,0.3853,0.8184,0.4087,0.8053,gelu,512,0.3,brick,2,1000,RMSprop,dropout,he_uniform
62,01/23/21-002318,01/23/21-002830,312.1586,18,0.3935,0.8144,0.4048,0.8052,gelu,512,0.3,triangle,2,1000,RMSprop,dropout,he_uniform
63,01/23/21-002830,01/23/21-003037,126.8356,26,0.4003,0.8106,0.4057,0.8050,gelu,512,0.3,brick,2,200,RMSprop,dropout,he_uniform
55,01/23/21-000446,01/23/21-001125,399.2471,21,0.3886,0.8178,0.4058,0.8050,gelu,512,0.5,triangle,2,1000,RMSprop,dropout,he_uniform


Result: no better network shape could be found

#### d) Best-Found Model: NN Structured

- Preprocessing = Yeo-Johnson + Standardization + PCA for numeric features; leave binary features as they are
- 3 hidden layers; [500, 350, 245] Units; GELU activation
- Dropout; Rate = 0.5
- HeUniform initialization
- RMSProp optimizer
- Batch Size: 512